In [34]:
import pandas as pd
pd.set_option("display.min_rows", 50)
pd.set_option("display.max_rows", 100)

In [35]:
gene_list = pd.read_table("./hgnc_complete_set.txt", header = 0).symbol.to_list()


/var/folders/j6/pp8gryw53252s4wz49ttk4gh0000gp/T/ipykernel_57250/3709584013.py:1: DtypeWarning: Columns (32,34,38,40,50) have mixed types. Specify dtype option on import or set low_memory=False.
  gene_list = pd.read_table("./hgnc_complete_set.txt", header = 0).symbol.to_list()


In [36]:
class solver: 
    def __init__(self, gene_list):
        '''take in a gene list, filter to only 5 letter names
        setup containers for incorrect letters.
        '''

        # define set of incorrect (grey) letters
        self.no_let = set()
        fivelet_genes = []
        for gene in gene_list:
            if len(gene) <= 5:
                fivelet_genes.append(gene)
        split_genes = []
        for gene in fivelet_genes:
            split_genes.append(list(gene))
        split_genes = pd.DataFrame(split_genes)
        split_genes = split_genes.fillna(" ")
        self.split_genes = split_genes
        self.possibilities = split_genes
        # container to track guesses
        self.guesses = []

    def __call__(self):
        return self.possibilities

    def guess(self, word, yellow=[], green=[], add_to_guess_list=True):
        '''
        first arg is a string, case sensitive of the guessed gene
        yellow and green are lists of positions of the yellow or green letters (1-based)
        add_to_guess_list is not needed for normal interaction, just for the self.redo function
        returns a list of possibilities
        '''
        if add_to_guess_list:
            self.guesses.append((word, yellow, green))
        wordlist = list(word)
        # require yellow letters to be in another position
        for position in yellow:
            pypos = position - 1
            self.possibilities = self.possibilities.loc[~self.possibilities[pypos].isin([wordlist[pypos]]),:]
            self.possibilities = self.possibilities.loc[self.possibilities.isin([wordlist[pypos]]).any(axis=1),:]
        # require green letters to be in same position
        for position in green:
            pypos = position - 1
            self.possibilities = self.possibilities.loc[self.possibilities[pypos].isin([wordlist[pypos]]),:]
        # add grey letters to blocked list
        for letter in wordlist:
            yellow_or_green_idx = yellow + green
            if wordlist.index(letter) + 1 not in yellow_or_green_idx:
                self.no_let.add(letter)
        for letter in self.no_let:
            self.possibilities = self.possibilities.loc[~self.possibilities.isin(self.no_let).any(axis=1),:]
        return self.possibilities 
    
    def redo(self):
        # allows rerunning the guesses after editing self.guesses if a mistake was made
        self.possibilities = self.split_genes
        for guess_set in self.guesses:
            self.guess(*guess_set, add_to_guess_list=False)
        return self.__call__()



In [37]:
mysolver = solver(gene_list)
mysolver()

,0,1,2,3,4
0,A,1,B,G,
1,A,1,C,F,
2,A,2,M,,
3,A,2,M,L,1
4,A,2,M,P,1
5,A,4,G,N,T
6,A,A,A,S,
7,A,A,C,S,
8,A,A,D,A,C
9,A,A,D,A,T


In [38]:
mysolver.guess("RPS3 ", yellow=[], green=[3])

,0,1,2,3,4
19,A,A,S,D,H
144,A,C,S,F,2
146,A,C,S,L,1
148,A,C,S,L,4
149,A,C,S,L,5
150,A,C,S,L,6
151,A,C,S,M,1
153,A,C,S,M,4
154,A,C,S,M,5
155,A,C,S,M,6


In [39]:
mysolver.guess("RPS3 ", yellow=[], green=[3])

,0,1,2,3,4
19,A,A,S,D,H
144,A,C,S,F,2
146,A,C,S,L,1
148,A,C,S,L,4
149,A,C,S,L,5
150,A,C,S,L,6
151,A,C,S,M,1
153,A,C,S,M,4
154,A,C,S,M,5
155,A,C,S,M,6


In [40]:
mysolver.guess("BEST4", yellow=[], green=[3])

,0,1,2,3,4
19,A,A,S,D,H
144,A,C,S,F,2
146,A,C,S,L,1
149,A,C,S,L,5
150,A,C,S,L,6
151,A,C,S,M,1
154,A,C,S,M,5
155,A,C,S,M,6
156,A,C,S,S,1
157,A,C,S,S,2


In [41]:
mysolver.guess("DISC1", yellow=[], green=[3])

,0,1,2,3,4
3221,F,A,S,L,G
3448,F,O,S,L,2
5214,K,A,S,H,5
6344,M,O,S,M,O
6908,N,H,S,L,2
9396,S,A,S,S,6
9590,S,G,S,M,2
11309,U,V,S,S,A
11325,V,A,S,H,2
11449,W,A,S,F,2


In [42]:
mysolver.guess("VASH2", yellow=[], green=[3])

,0,1,2,3,4
6344,M,O,S,M,O
